In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
# dataset
dataset_train_path = 'dataset1k'

## Importing necessary libraries

In [3]:
from numpy import load
import numpy as np
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
import pandas as pd
pd.set_option('display.max_rows', 2200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from shutil import copyfile
import string
import random
import os
from os import listdir
from os.path import isdir

embedder = FaceNet()

## Finding Duplicate Images

In [4]:
train_images_path = []
trainy =[]

def get_duplicate_images(directory, matching_cutoff_threshold = 0.8, threshold=0.95):

    embedding = []
    for subdir in listdir(directory):
        path = directory + '/' + subdir
        trainy.append(subdir)
        train_images_path.append(path)
        embedding.append(embedder.extract(path,threshold=threshold)[0]['embedding'])
    
    features_dataset = np.array(embedding)

    savez_compressed('./faces-train_embeddings.npz', features_dataset, trainy, train_images_path)

    feature = load('./faces-train_embeddings.npz')
    img_dup = pd.DataFrame([],columns=['Image_label','Duplicate_image','similarity_score'])
    for id1,feature1 in enumerate(zip(feature['arr_0'], feature['arr_1'], feature['arr_2'])):
        for id2,feature2 in enumerate(zip(feature['arr_0'], feature['arr_1'], feature['arr_2'])):
            if id2>id1:
                image_1,label_1,path_1 = feature1[0],feature1[1],feature1[2]
                image_2,label_2,path_2 = feature2[0],feature2[1],feature2[2]

                similarity_score = cosine_similarity(image_1.reshape(1, -1),
                                                     image_2.reshape(1, -1))
                if similarity_score > matching_cutoff_threshold:
                    img_dup.loc[len(img_dup)]= [label_1,label_2,similarity_score[0][0]]
    return img_dup

In [5]:
df_dup = get_duplicate_images(dataset_train_path)
df_dup

,Image_label,Duplicate_image,similarity_score
0,134.jpg,293.jpg,0.861108
1,146.jpg,230.jpg,0.885374
2,15.jpg,714.jpg,0.911921
3,171.jpg,657.jpg,0.811181
4,198.jpg,78.jpg,0.876593
5,20.jpg,665.jpg,0.827342
6,203.jpg,36.jpg,0.819869
7,205.jpg,637.jpg,0.815889
8,21.jpg,717.jpg,0.844782
9,23.jpg,989.jpg,0.959185


In [6]:
#export results to .csv file
# df_dup.to_csv('Duplicate_images.csv', index = None, header=True)

In [7]:
#save dupl images to folder 
# image_org = set(df_dup['Image_path'])
# image_dup = set(df_dup['Duplicate_path'])
# total_image = image_org.union(image_dup)

# generate random folder name to save original and duplicate
# foldername = ''.join(random.choice(string.ascii_uppercase) for i in range(5)) + '_OutPut'
# newdir = './' + foldername

# assert os.path.exists(newdir) == False , 'Folder Already Exist, Please Create new folder'

# create a folder
# os.makedirs(newdir)

# for image_path in total_image:    
#     dest_path = os.path.join(newdir,image_path.split('/')[-1])
#     # copy image to dest_path
#     _ = copyfile(image_path,dest_path)

In [8]:
import gradio as gr

In [9]:
face = gr.Interface(
    get_duplicate_images, 
    [
     gr.inputs.Textbox(lines = 2, placeholder = "please enter input directory path...."),
    ],

    gr.outputs.Dataframe(),
    # theme= 'dark-seafoam',
    css= """
    body {background-color : grey}
    """
)
face.launch()

Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x20bc7a9de80>,
 'http://127.0.0.1:7860/',
 None)